# Wisconsin Breast Cancer Dataset

In [1]:
import datetime
from sklearn import datasets
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models

## Load the data

In [3]:
bunch = datasets.load_breast_cancer()
features = bunch['feature_names']
features

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [4]:
X_all = pd.DataFrame(bunch['data'],columns=features)
y_all = pd.Series(bunch['target'])
X_all.describe()

mean radius  mean texture  mean perimeter    mean area  \
count   569.000000    569.000000      569.000000   569.000000   
mean     14.127292     19.289649       91.969033   654.889104   
std       3.524049      4.301036       24.298981   351.914129   
min       6.981000      9.710000       43.790000   143.500000   
25%      11.700000     16.170000       75.170000   420.300000   
50%      13.370000     18.840000       86.240000   551.100000   
75%      15.780000     21.800000      104.100000   782.700000   
max      28.110000     39.280000      188.500000  2501.000000   

       mean smoothness  mean compactness  mean concavity  mean concave points  \
count       569.000000        569.000000      569.000000           569.000000   
mean          0.096360          0.104341        0.088799             0.048919   
std           0.014064          0.052813        0.079720             0.038803   
min           0.052630          0.019380        0.000000             0.000000   
25%           0.086370          0.064920        0.029560             0.020310   
50%           0.095870          0.092630        0.061540             0.033500   
75%           0.105300          0.130400        0.130700             0.074000   
max           0.163400          0.345400        0.426800             0.201200   

       mean symmetry  mean fractal dimension  ...  worst radius  \
count     569.000000              569.000000  ...    569.000000   
mean        0.181162                0.062798  ...     16.269190   
std         0.027414                0.007060  ...      4.833242   
min         0.106000                0.049960  ...      7.930000   
25%         0.161900                0.057700  ...     13.010000   
50%         0.179200                0.061540  ...     14.970000   
75%         0.195700                0.066120  ...     18.790000   
max         0.304000                0.097440  ...     36.040000   

       worst texture  worst perimeter   worst area  worst smoothness  \
count     569.000000       569.000000   569.000000        569.000000   
mean       25.677223       107.261213   880.583128          0.132369   
std         6.146258        33.602542   569.356993          0.022832   
min        12.020000        50.410000   185.200000          0.071170   
25%        21.080000        84.110000   515.300000          0.116600   
50%        25.410000        97.660000   686.500000          0.131300   
75%        29.720000       125.400000  1084.000000          0.146000   
max        49.540000       251.200000  4254.000000          0.222600   

       worst compactness  worst concavity  worst concave points  \
count         569.000000       569.000000            569.000000   
mean            0.254265         0.272188              0.114606   
std             0.157336         0.208624              0.065732   
min             0.027290         0.000000              0.000000   
25%             0.147200         0.114500              0.064930   
50%             0.211900         0.226700              0.099930   
75%             0.339100         0.382900              0.161400   
max             1.058000         1.252000              0.291000   

       worst symmetry  worst fractal dimension  
count      569.000000               569.000000  
mean         0.290076                 0.083946  
std          0.061867                 0.018061  
min          0.156500                 0.055040  
25%          0.250400                 0.071460  
50%          0.282200                 0.080040  
75%          0.317900                 0.092080  
max          0.663800                 0.207500  

[8 rows x 30 columns]

In [5]:
print('Count zero',np.count_nonzero(y_all==0),'Malignant')
print('Count ones',np.count_nonzero(y_all),'Benign')

Count zero 212 Malignant
Count ones 357 Benign


## Partition the data: train vs test

In [6]:
print(datetime.datetime.now())
Xtrain,Xtest,ytrain,ytest = train_test_split(X_all, y_all.ravel()) 
        # ,random_state=42) # add this for reproducibility
print('Xtrain',Xtrain.shape,'ytrain',ytrain.shape,'non-zero:',np.count_nonzero(ytrain))
print('Xtest',Xtest.shape,'ytest',ytest.shape,'non-zero:',np.count_nonzero(ytest))

2022-06-06 09:18:28.980685
Xtrain (426, 30) ytrain (426,) non-zero: 271
Xtest (143, 30) ytest (143,) non-zero: 86


## Get cross-validation accuracy
Partition the training data into 5 equal-size groups.
For i = 1 to 5, hold group i for validation,
train the model on the other groups,
and evaluate the model on group i.

In [7]:
print(datetime.datetime.now())
model1=RandomForestClassifier()
cv_scores = cross_val_score(model1, Xtrain, ytrain, cv=5)
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-06 09:18:28.985512
[0.95348837 0.91764706 0.91764706 0.96470588 0.98823529]
mean 0.9483 +/- 0.0275


## Re-train the model. Evaluate on test data.
Whereas cross-validation trained on 80% of the training data (5 times),
now train on 100% of the training data.
Test on the test data.
Warning: Never use the test results to improve the model.

In [8]:
print(datetime.datetime.now())
model2=RandomForestClassifier()
model2.fit(Xtrain,ytrain)

2022-06-06 09:18:29.388855


RandomForestClassifier()

In [9]:
print(datetime.datetime.now())
ypred = model2.predict(Xtest)
matches = np.count_nonzero(ytest==ypred)
accuracy = 100.0 * matches / len(ypred) 
accuracy

2022-06-06 09:18:29.477368


97.9020979020979

In [10]:
print(datetime.datetime.now())
confusion = confusion_matrix(ytest, ypred)
confusion

2022-06-06 09:18:29.487110


array([[56,  1],
       [ 2, 84]])

## Rank features by importance
Warning: Random forests use random sampling.
We get a different ranking every time we build a new classifier.

In [11]:
def important_features(model):
    names = model.feature_names_in_
    importances = model.feature_importances_
    pairs = np.column_stack( (names,importances) )
    top_array = sorted(pairs, key = lambda e:e[1], reverse=True)
    top_list = []
    for i in top_array:
         top_list.append((i[1],i[0]))  # 0=feature_name, 1=importance
    top_df = pd.DataFrame(top_list)
    return top_df
print(datetime.datetime.now())
top_features = important_features(model2)
top_features[:10]

2022-06-06 09:18:29.491265


0                     1
0  0.150718       worst perimeter
1  0.129410  worst concave points
2  0.108985          worst radius
3  0.106031            worst area
4  0.080374   mean concave points
5  0.064665             mean area
6  0.051304       worst concavity
7  0.037349        mean perimeter
8  0.032951        mean concavity
9  0.032639           mean radius